In [4]:
# 导入库
import numpy as np
import pandas as pd
from   datetime import datetime
from tqdm.notebook import tqdm
import os
import glob
import pysentiment2 as ps
import nltk
from   nltk.sentiment.vader import SentimentIntensityAnalyzer
from   IPython.core.display import HTML

from transformers import pipeline
from transformers import AutoTokenizer, AutoModelForSequenceClassification

import torch

In [2]:
## 设置情感分析工具

# 下载Vader情感分析工具的词典
nltk.download('vader_lexicon')
# 初始化Loughran-McDonald情感分析模型
lm    = ps.LM()
# 初始化HIV-4情感分析模型
hiv4  = ps.HIV4()
# 初始化Vader情感分析模型
vader = SentimentIntensityAnalyzer()

# 如果有可用的第一块GPU则使用，否则使用CPU
device = torch.device("mps"if torch.backends.mps.is_available()else "cpu")

# 初始化FinBERT情感分析模型的tokenizer
finbert_tokenizer  = AutoTokenizer.from_pretrained("ProsusAI/finbert")
# 从FinBERT预训练模型加载模型
finbert_model      = AutoModelForSequenceClassification.from_pretrained("ProsusAI/finbert")
# 配置FinBERT情感分析模型的运行环境（GPU or CPU）
finbert_classifier = pipeline("sentiment-analysis", model = finbert_model, tokenizer = finbert_tokenizer, device = device)

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/wanghs/nltk_data...


Xformers is not installed correctly. If you want to use memorry_efficient_attention to accelerate training use the following command to install Xformers
pip install xformers.


In [16]:
def sentimentAnalysisLMD(text):
    tokens = lm.tokenize(text) #tokenize the text into relevant words in the LM dict
    score = lm.get_score(tokens) #score dict seperates into: NEG, POS, POLARITY, SUBJECTIVITY
    return score

def sentimentAnalysisHIV4(text):
    tokens = hiv4.tokenize(text)
    score = hiv4.get_score(tokens)
    return score

def sentimentAnalysisVader(df):
    df_vader = pd.DataFrame(df, columns=['Date', 'Text'])
    scores = [vader.polarity_scores(text) for text in tqdm(df_vader['Text'], desc="Processing with VADER")]
    scores_vader = pd.DataFrame(scores).add_prefix("vader_")
    df_vader = df_vader.join(scores_vader, rsuffix='_right')
    return df_vader['vader_compound']

def sentimentAnalysisFinBert(df):
    df_finbert = pd.DataFrame(df, columns=['Date', 'Text'])
    score = []
    for text in tqdm(df_finbert['Text'], desc="Processing with FinBERT"):
        classified = finbert_classifier(text, truncation=True)[0]
        if classified['label'] == "negative":
            score.append(classified['score']*(-1))
        elif classified['label'] == "positive":
            score.append(classified['score'])
        else:
            score.append(0)
    return score

In [ ]:
lst_files = []

Path = "../Data/Reuters_NER/*.csv"
count = 0

for fname in glob.glob(Path):
    lst_files.append(fname)

lst_files = sorted(lst_files)

for file in lst_files:
    file_name = os.path.basename(file)  # Get the file name from the full file path
    country_name = os.path.splitext(file_name)[0]  # Remove the file extension

    # 对单个国家的数据进行情感分析
    # if country_name == 'Finland':
    # Check if the file already exists
    if os.path.isfile('../Data/SentimentScores/{0}.csv'.format(country_name)):
        print("File already exists: {0}.csv".format(country_name))
    else:
        print(file)
        df = pd.read_csv(file)

        if df.shape[0] != 0:
            df = df.drop_duplicates(['Date','Headline'], keep='last')
            df = df.drop(['Unnamed: 0', 'Headline'], axis = 1)
            df['Text'] = df['Text'].astype(str)
            df['Date'] = pd.to_datetime(df['Date'], utc=True).dt.date
            df.reset_index(drop=True, inplace=True)

            # LMD + HIV4
            df_LMD_HIV4 = pd.DataFrame(df, columns=['Date', 'Text'])
            # df_LMD_HIV4["scoreLMD"] = df_LMD_HIV4["Text"].apply(sentimentAnalysisLMD)
            # df_LMD_HIV4["scoreHIV4"] = df_LMD_HIV4["Text"].apply(sentimentAnalysisHIV4)
            # 使用tqdm在处理sentiment时显示进度条
            df_LMD_HIV4["scoreLMD"] = [sentimentAnalysisLMD(text) for text in tqdm(df_LMD_HIV4["Text"], desc="Processing with LMD")]
            df_LMD_HIV4["scoreHIV4"] = [sentimentAnalysisHIV4(text) for text in tqdm(df_LMD_HIV4["Text"], desc="Processing with HIV4")]
            df_LMD_HIV4 = pd.concat([df_LMD_HIV4.drop(["scoreLMD"], axis=1), df_LMD_HIV4["scoreLMD"].apply(pd.Series).add_prefix("LMD_")], axis=1)
            df_LMD_HIV4 = pd.concat([df_LMD_HIV4.drop(["scoreHIV4"], axis=1), df_LMD_HIV4["scoreHIV4"].apply(pd.Series).add_prefix("HIV4_")], axis=1)
            df[['LMD_Polarity', 'HIV4_Polarity']] = df_LMD_HIV4[['LMD_Polarity', 'HIV4_Polarity']]

            # VADER
            df['Vader_Polarity'] = sentimentAnalysisVader(df)

            # FINBERT
            df['FinBert_Polarity'] = sentimentAnalysisFinBert(df)

            # 储存一下df的原始数据
            df_original = df.copy()
            df['Country'] = country_name
            # Find the average Polarity for each Dictionary, per given date
            df = df.groupby('Date').mean(numeric_only=True).reset_index()
            df = df.sort_values(by = ['Date'], ascending = True)
            idx = pd.date_range('2012-01-01', '2022-12-31')
            df = df.set_index(['Date']).reindex(idx, fill_value=np.nan).rename_axis('Date').reset_index()

            df['Country'] = country_name
            df= df[['Date', 'Country', 'LMD_Polarity', 'HIV4_Polarity', 'Vader_Polarity', 'FinBert_Polarity']]

            count = count + 1
            print(count)
            df.to_csv('../Data/SentimentScores/{0}.csv'.format(country_name))
            # 保存df的原始数据
            df_original.to_csv('../Data/SentimentScores/{0}_original.csv'.format(country_name))


../Data/Reuters_NER/Australia.csv


Processing with LMD:   0%|          | 0/9234 [00:00<?, ?it/s]

Processing with HIV4:   0%|          | 0/9234 [00:00<?, ?it/s]

Processing with VADER:   0%|          | 0/9234 [00:00<?, ?it/s]

Processing with FinBERT:   0%|          | 0/9234 [00:00<?, ?it/s]

...

16
../Data/Reuters_NER/Sweden.csv


Processing with LMD:   0%|          | 0/2202 [00:00<?, ?it/s]

Processing with HIV4:   0%|          | 0/2202 [00:00<?, ?it/s]

Processing with VADER:   0%|          | 0/2202 [00:00<?, ?it/s]

Processing with FinBERT:   0%|          | 0/2202 [00:00<?, ?it/s]

In [5]:
# 展示每个国家的情感分析数据
lst_files = []
path = "../Data/SentimentScores/*.csv"

for fname in glob.glob(path):
    # 检查文件名是否以 "_original.csv" 结尾
    if not fname.endswith("_original.csv"):
        lst_files.append(fname)

lst_files = sorted(lst_files)

# 读取每个国家数据, 并输出每个数据的前5行
for file in lst_files:
    file_name = os.path.basename(file)  # Get the file name from the full file path
    country_name = os.path.splitext(file_name)[0]  # Remove the file extension
    df = pd.read_csv(file)
    print(country_name)
    print(df.head())
    print('\n')


Australia
   Unnamed: 0        Date    Country  LMD_Polarity  HIV4_Polarity  \
0           0  2012-01-01  Australia           NaN            NaN   
1           1  2012-01-02  Australia           NaN            NaN   
2           2  2012-01-03  Australia           NaN            NaN   
3           3  2012-01-04  Australia           NaN            NaN   
4           4  2012-01-05  Australia           NaN            NaN   

   Vader_Polarity  FinBert_Polarity  
0             NaN               NaN  
1             NaN               NaN  
2             NaN               NaN  
3             NaN               NaN  
4             NaN               NaN  


Canada
   Unnamed: 0        Date Country  LMD_Polarity  HIV4_Polarity  \
0           0  2012-01-01  Canada           NaN            NaN   
1           1  2012-01-02  Canada           NaN            NaN   
2           2  2012-01-03  Canada           NaN            NaN   
3           3  2012-01-04  Canada           NaN            NaN   
4       

In [6]:
# 展示每个国家的原始情感数据
lst_files = []
path = "../Data/SentimentScores/*.csv"

for fname in glob.glob(path):
    # 检查文件名是否以 "_original.csv" 结尾
    if fname.endswith("_original.csv"):
        lst_files.append(fname)

lst_files = sorted(lst_files)

# 读取每个国家数据, 并输出每个数据的前5行
for file in lst_files:
    file_name = os.path.basename(file)  # Get the file name from the full file path
    country_name = os.path.splitext(file_name)[0]  # Remove the file extension
    df = pd.read_csv(file)
    print(country_name)
    print(df.head())
    print('\n')

Australia_original
   Unnamed: 0        Date                                               Text  \
0           0  2021-01-20  China’s imports of Australian copper ore crash...   
1           1  2021-01-28  First flashes of Brexit trade trouble appear i...   
2           2  2021-02-09  Stocks, oil rise in Asia after U.S. records, d...   
3           3  2021-02-22  Future proofing: Australia’s gas networks look...   
4           4  2021-02-26  Take Five: Policymakers under pressure. 1/TANT...   

   LMD_Polarity  HIV4_Polarity  Vader_Polarity  FinBert_Polarity    Country  
0     -1.000000       0.214286         -0.9485         -0.972263  Australia  
1     -0.411765       0.050847         -0.6597         -0.969801  Australia  
2     -0.066667       0.333333          0.9799          0.595929  Australia  
3      0.400000       0.449275          0.9964          0.000000  Australia  
4     -0.666667      -0.012658          0.9554          0.561960  Australia  


Canada_original
   Unnamed: 0 

In [8]:
# 合并所有国家的情感分析数据
lst_files = []
path = "../Data/SentimentScores/*.csv"

for fname in glob.glob(path):
    # 检查文件名是否以 "_original.csv" 结尾
    if not fname.endswith("_original.csv"):
        lst_files.append(fname)

lst_files = sorted(lst_files)

df = pd.concat(map(pd.read_csv, lst_files), ignore_index=True)

# Sort by Date, Country; drop redundant columns
df = df.sort_values(by = ['Date', 'Country'], ascending = True)
df = df.drop(['Unnamed: 0'], axis = 1)

# Align Sentiment with the following day's Return
df['Date'] = pd.to_datetime(df['Date'])
df = df[~(df['Date'] < '2012-01-01')]
# df['Date'] = df["Date"] + BusinessDay()

# 展示数据
# df

df.to_csv(r'../Data/{0}.csv'.format('Merged Sentiment Reuters'))
